In [1]:
from openai import OpenAI

In [1]:
import pickle
with open('elastic_search_data.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

In [2]:
for transcript in transcripts:
    transcript.pop('search_vector', None) 

In [4]:
transcripts[0]

{'vid_id': 'Q75JgLEXMsM',
 'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
 'timecode': '00:00',
 'text': "hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I'll quickly open one of them and in these documents we have frequently asked questi

In [78]:
def make_prompt(transcript):
    return f"""
You are an expert at generating potential questions based on course content. This course teaches Retrieval Augmented Generation (RAG) and Large Language 
Models (LLMs). Please generate three potential questions a student might ask, specifically where the given record would be the most relevant to provide 
answers. Return the original dictionary with a new field 'student_questions' added. The field should contain a list of three questions as 
'student_questions': [question1, question2, question3]. Return the dictionary in JSON format. Not python. JSON format and just return the dictionary
in JSON FORMAT

Here is the record for which to generate the questions:

DICTIONARY:
{transcript}
"""

In [5]:
prompt = make_prompt(transcripts[0])

NameError: name 'make_prompt' is not defined

In [7]:
prompt

'\nYou are an expert at generating potential questions based on course content. This course teaches Retrieval Augmented Generation (RAG) and Large Language \nModels (LLMs). Please generate three potential questions a student might ask, specifically where the given record would be the most relevant to provide \nanswers. Return the original dictionary with a new field \'student_questions\' added. The field should contain a list of three questions as \n\'student_questions\': [question1, question2, question3].\n\nHere is the record for which to generate the questions:\n\nDICTIONARY:\n{\'vid_id\': \'Q75JgLEXMsM\', \'title\': \'LLM Zoomcamp 1.1 - Introduction to LLM and RAG\', \'timecode\': \'00:00\', \'text\': "hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I\'ll shortly t

In [8]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
from openai import OpenAI

client = OpenAI()
def make_recs(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    # Extract response content
    response_content = response.choices[0].message

    # Optionally, print or process the response
    return response_content

In [10]:
test = make_recs(prompt)

In [14]:
response = test.content

In [21]:
import json
import re
start_index = response.index("{")
end_index = response.rindex("}") + 1
dict_str = response[start_index:end_index]

# Step 2: Replace the outer single quotes with double quotes using regex, preserving inner quotes
dict_str = re.sub(r"(?<!\\)'", '"', dict_str)

# Step 3: Try to load the dictionary using the json module
try:
    parsed_dict = json.loads(dict_str)
    print(parsed_dict)
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

Error parsing JSON: Expecting ',' delimiter: line 5 column 285 (char 403)


In [36]:
broken_dict = response.split('```')[1].split('python')[1].replace('\n','').split(',')

In [63]:
fixer = f"""
Can you format this into a proper python dictionary and just return it as a dictionary? Return JUST THE DICTIONARY without any extra words:
I don't want anything except the dictionary I don't want code blocks with ```. Just the formatted dictionary. Additionally can you remove any single or
double quotes and commas in the values which may cause issues when using ast.literal_eval?

 {broken_dict}
"""

In [64]:
import ollama
def ask_ollama(prompt):
    response = ollama.chat(model='llama3', messages=[
    {
        'role': 'user',
        'content': f"{prompt}",
    },
    ])
    return response['message']['content']

In [65]:
fixed_dict = ask_ollama(fixer)

In [66]:
import ast
ast.literal_eval(fixed_dict)

{'vid_id': 'Q75JgLEXMsM',
 'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
 'timecode': '00:00',
 'text': 'hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I\'ll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I\'ll quickly open one of them and in these documents we have frequently asked ques

In [6]:
with open('elastic_search_data.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

for transcript in transcripts:
    transcript.pop('search_vector', None) 

In [7]:
len(transcripts)

279

In [8]:
with open('OpenAI_responses.pkl', 'rb') as infile:
    data = pickle.load(infile)

In [9]:
len(data)

279

In [10]:
count = 0
for i in data:
    
    try:
        i.content.split('```')[1].split('python')[1].replace('\n','').split(',')
    except Exception as e:
        count +=1
print(count)

279


In [17]:
import ast
ast.literal_eval(data[0].content)

{'vid_id': 'Q75JgLEXMsM',
 'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
 'timecode': '00:00',
 'text': "hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I'll quickly open one of them and in these documents we have frequently asked questi

In [11]:
import ast
example_questions = []
for rec in data:
    example_questions.append(ast.literal_eval(rec.content))

In [12]:
new_data = []

# Loop through each dictionary in the data list
for item in example_questions:
    # Extract the common fields
    base_dict = {key: item[key] for key in item if key != 'student_questions'}
    
    # For each question, create a new dictionary with the question added
    for question in item['student_questions']:
        # Create a new dictionary with the common fields and add the question
        new_dict = base_dict.copy()
        new_dict['student_question'] = question
        
        # Append the new dictionary to the result list
        new_data.append(new_dict)



In [13]:
len(new_data)

837

In [98]:
279 * 3

837

In [100]:
with open('ground_truth.pkl','wb') as outfile:
    pickle.dump(new_data, outfile)